In [1]:
!wget http://www.mpi-sws.org/~cristian/data/cornell_movie_dialogs_corpus.zip

--2022-09-03 21:44:26--  http://www.mpi-sws.org/~cristian/data/cornell_movie_dialogs_corpus.zip
Resolving www.mpi-sws.org (www.mpi-sws.org)... 139.19.205.208
Connecting to www.mpi-sws.org (www.mpi-sws.org)|139.19.205.208|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://www.mpi-sws.org/~cristian/data/cornell_movie_dialogs_corpus.zip [following]
--2022-09-03 21:44:27--  https://www.mpi-sws.org/~cristian/data/cornell_movie_dialogs_corpus.zip
Connecting to www.mpi-sws.org (www.mpi-sws.org)|139.19.205.208|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://people.mpi-sws.org/~cristian/data/cornell_movie_dialogs_corpus.zip [following]
--2022-09-03 21:44:27--  https://people.mpi-sws.org/~cristian/data/cornell_movie_dialogs_corpus.zip
Resolving people.mpi-sws.org (people.mpi-sws.org)... 139.19.206.131
Connecting to people.mpi-sws.org (people.mpi-sws.org)|139.19.206.131|:443... connected.
HTTP request sent, awai

In [2]:
!unzip "/content/cornell_movie_dialogs_corpus.zip" -d "cornell_movie_dialogs_corpus/"

Archive:  /content/cornell_movie_dialogs_corpus.zip
replace cornell_movie_dialogs_corpus/cornell movie-dialogs corpus/.DS_Store? [y]es, [n]o, [A]ll, [N]one, [r]ename: غ
error:  invalid response [غ]
replace cornell_movie_dialogs_corpus/cornell movie-dialogs corpus/.DS_Store? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: cornell_movie_dialogs_corpus/cornell movie-dialogs corpus/.DS_Store  
replace cornell_movie_dialogs_corpus/__MACOSX/cornell movie-dialogs corpus/._.DS_Store? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: cornell_movie_dialogs_corpus/__MACOSX/cornell movie-dialogs corpus/._.DS_Store  
replace cornell_movie_dialogs_corpus/cornell movie-dialogs corpus/chameleons.pdf? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: cornell_movie_dialogs_corpus/cornell movie-dialogs corpus/chameleons.pdf  
replace cornell_movie_dialogs_corpus/__MACOSX/cornell movie-dialogs corpus/._chameleons.pdf? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: cornell_movie_dialogs_co

In [3]:
from __future__ import print_function
import sys
import os
import pandas as pd
import numpy as np
import re

In [4]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [5]:
lines = open("/content/cornell_movie_dialogs_corpus/cornell movie-dialogs corpus/movie_lines.txt", encoding="utf-8", errors="ignore")
lines = lines.read().split('\n')

conv_lines = open("/content/cornell_movie_dialogs_corpus/cornell movie-dialogs corpus/movie_conversations.txt", encoding="utf-8", errors="ignore")
conv_lines = conv_lines.read().split("\n")

In [6]:
id2line = {}
for line in lines:
  _line = line.split(" +++$+++ ")
  if len(_line) == 5:
    id2line[_line[0]] = _line[4]

In [7]:
convs = []
for line in conv_lines[:-1]:
    _line = line.split(" +++$+++ ")[-1][1:-1]
    _line = _line.replace("'","").replace(" ","")
    convs.append(_line.split(','))

In [8]:
for k in convs[0]:
  print(k, id2line[k])

L194 Can we make this quick?  Roxanne Korrine and Andrew Barrett are having an incredibly horrendous public break- up on the quad.  Again.
L195 Well, I thought we'd start with pronunciation, if that's okay with you.
L196 Not the hacking and gagging and spitting part.  Please.
L197 Okay... then how 'bout we try out some French cuisine.  Saturday?  Night?


In [9]:
questions = []
answers = []
for conv in convs:
  for i in range(len(conv)-1):
    questions.append(id2line[conv[i]])
    answers.append(id2line[conv[i+1]])

print(len(questions))
print(len(answers))

221616
221616


In [10]:
def clean_text(text):
  text = text.lower()
  text = re.sub(r"i'm", "i am", text)
  text = re.sub(r"he's", "he is", text)
  text = re.sub(r"she's", "she is", text)
  text = re.sub(r"it's", "it is", text)
  text = re.sub(r"that's", "that is", text)
  text = re.sub(r"what's", "that is", text)
  text = re.sub(r"where's", "where is", text)
  text = re.sub(r"how's", "how is", text)
  text = re.sub(r"\'ll", " will", text)
  text = re.sub(r"\'ve", " have", text)
  text = re.sub(r"\'re", " are", text)
  text = re.sub(r"\'d", " would", text)
  text = re.sub(r"\'re", " are", text)
  text = re.sub(r"won't", "will not", text)
  text = re.sub(r"can't", "cannot", text)
  text = re.sub(r"n't", " not", text)
  text = re.sub(r"n'", "ng", text)
  text = re.sub(r"'bout", "about", text)
  text = re.sub(r"'til", "until", text)
  text = re.sub(r"[-()\"#/@;:<>{}`+=~|.!?,]", "", text)
  text = " ".join(text.split())
  return text

In [11]:
clean_questions = []
for question in questions:
  clean_questions.append(clean_text(question))

In [12]:
clean_answers = []
for answer in answers:
  clean_answers.append(clean_text(answer))

In [13]:
lengths = []
for question in clean_questions:
  lengths.append(len(question.split()))

for answer in clean_answers:
  lengths.append(len(answer.split()))

lengths = pd.DataFrame(lengths, columns=["text length"])
print(np.percentile(lengths, 80))
print(np.percentile(lengths, 85))
print(np.percentile(lengths, 90))
print(np.percentile(lengths, 95))

16.0
19.0
24.0
32.0


In [14]:
min_line_length = 2
max_line_length = 20

short_questions_temp = []
short_answers_temp = []

for i, question  in enumerate(clean_questions):
  if len(question.split()) >= min_line_length and len(question.split()) <= max_line_length:
    short_questions_temp.append(question)
    short_answers_temp.append(clean_answers[i])

In [15]:
short_questions = []
short_answers = []

for i, answer in enumerate(short_answers_temp):
  if len(answer.split()) >= min_line_length and len(answer.split()) <= max_line_length:
    short_answers.append(answer)
    short_questions.append(short_questions_temp[i])

print(len(short_questions))
print(len(short_answers))

138335
138335


In [16]:
rand = np.random.randint(1, len(short_questions))

for i in range(rand, rand+3):
  print(short_questions[i])
  print(short_answers[i])
  print()

i knew his greatgrandfather
you are insane

you are insane
no seriously we used to shoot pool together in rangoon

no seriously we used to shoot pool together in rangoon
how do you do it kahn how do you live so full of life for so long



In [17]:
num_samples = 30000
short_questions = short_questions[:num_samples]
short_answers = short_answers[:num_samples]


In [18]:
short_questions_token = [nltk.word_tokenize(sent) for sent in short_questions]

short_answers_token = [nltk.word_tokenize(sent) for sent in short_answers]

In [19]:
data_size = len(short_questions_token)

training_input = short_questions_token[:round(data_size*0.8)]
training_input = [tr_input[::-1] for tr_input in training_input]
training_output = short_answers_token[:round(data_size*0.8)]

In [20]:
validation_input = short_questions_token[round(data_size*(80/100)):]
validation_input  = [val_input[::-1] for val_input in validation_input] #reverseing input seq for better performance
validation_output = short_questions_token[round(data_size*(80/100)):]

In [21]:
print('training size', len(training_input))
print('validation size', len(validation_input))

training size 24000
validation size 6000


In [22]:
vocab = {}

for question in short_questions_token:
  for word in question:
    if word not in vocab:
      vocab[word] = 1
    else:
      vocab[word] += 1

for answer in short_answers_token:
  for word in answer:
    if word not in vocab:
      vocab[word] = 1
    else:
      vocab[word] += 1

In [23]:
threshold = 15
count = 0

for k, v in vocab.items():
  if v >= threshold:
    count += 1

In [24]:
print("Size of total vocab:", len(vocab))
print("Size of vocab we will use:", count)

Size of total vocab: 16655
Size of vocab we will use: 1935


In [25]:
WORD_CODE_START = 1
WORD_CODE_PADDING = 0

word_num = 2

encoding = {}
decoding = {1: "START"}

for word, count in vocab.items():
  if count >= threshold:
    encoding[word] = word_num
    decoding[word_num] = word
    word_num += 1

print("No. of vocab used:", word_num)

No. of vocab used: 1937


In [26]:
decoding[len(encoding)+2] = '<UNK>'
encoding['<UNK>'] = len(encoding)+2

In [27]:
dict_size = word_num+1
dict_size

1938

In [28]:
def transform(encoding, data, vector_size=20):
  transformed_data = np.zeros(shape=(len(data), vector_size))
  for i in range(len(data)):
    for j in range(min(len(data[i]), vector_size)):
      try:
        transformed_data[i][j] = encoding[data[i][j]]
      except:
        transformed_data[i][j] = encoding["<UNK>"]

  return transformed_data

In [29]:
INPUT_LENGTH = 20
OUTPUT_LENGTH = 20

encoded_training_input = transform(encoding, training_input, 
                                   vector_size=INPUT_LENGTH)

encoded_training_output = transform(encoding, training_input, 
                                   vector_size=OUTPUT_LENGTH)

print('encoded_training_input', encoded_training_input.shape)
print('encoded_training_output', encoded_training_output.shape)

encoded_training_input (24000, 20)
encoded_training_output (24000, 20)


In [30]:
encoded_validation_input = transform(
    encoding, validation_input, vector_size=INPUT_LENGTH)
encoded_validation_output = transform(
    encoding, validation_output, vector_size=OUTPUT_LENGTH)

print('encoded_validation_input', encoded_validation_input.shape)
print('encoded_validation_output', encoded_validation_output.shape)

encoded_validation_input (6000, 20)
encoded_validation_output (6000, 20)


In [31]:
import tensorflow as tf
tf.keras.backend.clear_session()

In [32]:
from keras.layers import Input, Embedding, LSTM, TimeDistributed, Dense, Bidirectional
from keras.layers import Activation, dot, concatenate
from keras.models import Model, load_model

In [33]:
encoder_input = Input(shape=(INPUT_LENGTH,))
decoder_input = Input(shape=(OUTPUT_LENGTH,))

In [34]:
encoder = Embedding(dict_size, 128, input_length=INPUT_LENGTH, mask_zero=True)(encoder_input)
encoder = LSTM(512, return_sequences=True, unroll=True)(encoder)
encoder_last = encoder[:,-1,:]

print("encoder", encoder)
print("encoder_last", encoder_last)

encoder KerasTensor(type_spec=TensorSpec(shape=(None, 20, 512), dtype=tf.float32, name=None), name='lstm/transpose_2:0', description="created by layer 'lstm'")
encoder_last KerasTensor(type_spec=TensorSpec(shape=(None, 512), dtype=tf.float32, name=None), name='tf.__operators__.getitem/strided_slice:0', description="created by layer 'tf.__operators__.getitem'")


In [35]:
decoder = Embedding(dict_size, 128, input_length=OUTPUT_LENGTH, mask_zero=True)(decoder_input)
decoder = LSTM(512, return_sequences=True, unroll=True)(decoder, initial_state=[encoder_last, encoder_last])
print('decoder', decoder)

decoder KerasTensor(type_spec=TensorSpec(shape=(None, 20, 512), dtype=tf.float32, name=None), name='lstm_1/transpose_2:0', description="created by layer 'lstm_1'")


In [36]:
attention = dot([decoder, encoder], axes=[2,2])
attention = Activation("softmax", name="attention")(attention)
print('attention', attention)

context = dot([attention, encoder], axes=[2,1])
print('context', context)

decoder_combined_context = concatenate([context, decoder])
print('decoder_combined_context', decoder_combined_context)

output = TimeDistributed(Dense(512, activation="tanh"))(decoder_combined_context)
output = TimeDistributed(Dense(dict_size, activation="softmax"))(output)
print("output", output)

attention KerasTensor(type_spec=TensorSpec(shape=(None, 20, 20), dtype=tf.float32, name=None), name='attention/Softmax:0', description="created by layer 'attention'")
context KerasTensor(type_spec=TensorSpec(shape=(None, 20, 512), dtype=tf.float32, name=None), name='dot_1/MatMul:0', description="created by layer 'dot_1'")
decoder_combined_context KerasTensor(type_spec=TensorSpec(shape=(None, 20, 1024), dtype=tf.float32, name=None), name='concatenate/concat:0', description="created by layer 'concatenate'")
output KerasTensor(type_spec=TensorSpec(shape=(None, 20, 1938), dtype=tf.float32, name=None), name='time_distributed_1/Reshape_1:0', description="created by layer 'time_distributed_1'")


In [37]:
model = Model(inputs=[encoder_input, decoder_input], outputs=[output])
model.compile(optimizer="adam", loss="binary_crossentropy")
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 20)]         0           []                               
                                                                                                  
 embedding (Embedding)          (None, 20, 128)      248064      ['input_1[0][0]']                
                                                                                                  
 input_2 (InputLayer)           [(None, 20)]         0           []                               
                                                                                                  
 lstm (LSTM)                    (None, 20, 512)      1312768     ['embedding[0][0]']              
                                                                                              

In [38]:
training_encoder_input = encoded_training_input
training_decoder_input = np.zeros_like(encoded_training_output)

training_decoder_input[:, 1:] = encoded_training_output[:, :-1]
training_decoder_input[:, 0] = WORD_CODE_START

training_decoder_output = np.eye(dict_size)[encoded_training_output.astype("int")]

In [39]:
validation_encoder_input = encoded_validation_input
validation_decoder_input = np.zeros_like(encoded_validation_output)

validation_decoder_input[:, 1:] = encoded_validation_output[:, :-1]
validation_decoder_input[:, 0] = WORD_CODE_START

validation_decoder_output = np.eye(dict_size)[encoded_validation_output.astype("int")]

In [ ]:
model.fit(x=[training_encoder_input, training_decoder_input], 
          y=[training_decoder_output],
          validation_data=([validation_encoder_input, validation_decoder_input], 
                           [validation_decoder_output]),
          batch_size=64,
          epochs=100)

model.save("model_attention.h5")

In [ ]:
def prediction(raw_input):
    clean_input = clean_text(raw_input)
    input_tok = [nltk.word_tokenize(clean_input)]
    input_tok = [input_tok[0][::-1]]  
    encoder_input = transform(encoding, input_tok, 20)
    decoder_input = np.zeros(shape=(len(encoder_input), OUTPUT_LENGTH))
    decoder_input[:,0] = WORD_CODE_START
    for i in range(1, OUTPUT_LENGTH):
        output = model.predict([encoder_input, decoder_input]).argmax(axis=2)
        decoder_input[:,i] = output[:,i]
    return output

In [ ]:
def decode(decoding, vector):
    text = ''
    for i in vector:
        if i == 0:
            break
        text += ' '
        text += decoding[i]
    return text

In [ ]:
for i in range(20):
    seq_index = np.random.randint(1, len(short_questions))
    output = prediction(short_questions[seq_index])
    print ('Q:', short_questions[seq_index])
    print ('A:', decode(decoding, output[0]))

In [ ]:
raw_input = input()
output = prediction(raw_input)
print (decode(decoding, output[0]))